# Задание 
Решить задачу перевода с помощью механизма внимания

1. Возьмите англо-русскую пару фраз  (https://www.manythings.org/anki/):
2. Обучите на них seq2seq with attention:
      *  На основе скалярного произведения
      *  На основе MLP
3.Оцените качество

In [9]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
device

device(type='cuda')

In [10]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2022-08-03 13:07:46--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14819554 (14M) [application/zip]
Saving to: ‘rus-eng.zip.3’

rus-eng.zip.3       100%[===================>]  14.13M  2.09MB/s    in 8.5s    

2022-08-03 13:07:55 (1.67 MB/s) - ‘rus-eng.zip.3’ saved [14819554/14819554]

Archive:  rus-eng.zip
replace rus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [11]:
!tail rus.txt

A mistake young people often make is to start learning too many languages at the same time, as they underestimate the difficulties and overestimate their own ability to learn them.	Ошибка, которую часто делают молодые, — начинают учить слишком много языков одновременно: они недооценивают трудности и переоценивают свои способности к изучению.	CC-BY 2.0 (France) Attribution: tatoeba.org #2783162 (catcher) & #5118905 (Wezel)
We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (CO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mar

In [12]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [13]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-Я.!?]+", r" ", s)
    return s

In [14]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    # lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
    #     read().strip().split('\n')
    lines = open('rus.txt', encoding='utf-8').read().strip().split('\n')


    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:-1]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [15]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [16]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'rus', True)

print(random.choice(pairs))

Reading lines...
Read 444587 sentence pairs
Trimmed to 25903 sentence pairs
Counting words...
Counted words:
rus 9706
eng 4150
['мы внутри .', 'we re inside .']


## Encoder

In [17]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Decoder scalar

In [18]:
class AttnDecoderRNN_Dot(nn.Module):
    def _print(self, text):
        if self.debug:
            print(text)

    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH, debug=False):
        super(AttnDecoderRNN_Dot, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.debug = debug

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        attn_weights = F.softmax(
            torch.bmm(hidden, encoder_outputs.squeeze(0).T.unsqueeze(0)).squeeze(0), dim=1)   
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        if self.debug:
            return
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Decoder MLP

In [19]:
class AttnDecoderRNN_MLP(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN_MLP, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        output = self.attn_combine(torch.cat((embedded[0], attn_applied[0]), 1)).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [20]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [21]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [22]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [23]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [24]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [25]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [26]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [27]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [28]:
hidden_size = 256

encoder_1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder_1 = AttnDecoderRNN_Dot(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder_1, attn_decoder_1, 75000, print_every=5000)

1m 6s (- 15m 36s) (5000 6%) 3.0793
2m 10s (- 14m 6s) (10000 13%) 2.5628
3m 13s (- 12m 54s) (15000 20%) 2.3320
4m 18s (- 11m 49s) (20000 26%) 2.0953
5m 21s (- 10m 42s) (25000 33%) 1.9103
6m 25s (- 9m 38s) (30000 40%) 1.7854
7m 29s (- 8m 33s) (35000 46%) 1.6978
8m 34s (- 7m 30s) (40000 53%) 1.6010
9m 38s (- 6m 25s) (45000 60%) 1.5247
10m 43s (- 5m 21s) (50000 66%) 1.4344
11m 47s (- 4m 17s) (55000 73%) 1.3613
12m 52s (- 3m 13s) (60000 80%) 1.2677
13m 58s (- 2m 8s) (65000 86%) 1.2342
15m 3s (- 1m 4s) (70000 93%) 1.1676
16m 9s (- 0m 0s) (75000 100%) 1.1350


In [29]:
evaluateRandomly(encoder_1, attn_decoder_1)

> мы скоро отсюда уидем .
= we are about to leave here .
< we re about to leave here . <EOS>

> я устал это слышать .
= i am tired of hearing it .
< i m tired of hearing that . <EOS>

> я студент университета .
= i m a student in a university .
< i m a a student . <EOS>

> она улыбнулась своему ребенку .
= she smiled at her baby .
< she smiled a good her her . <EOS>

> мы не планируем идти вместе .
= we re not planning to go together .
< we re not planning together together . <EOS>

> я упрямыи .
= i m stubborn .
< i m a . <EOS>

> он романист и поэт .
= he is a novelist and poet .
< he is a a and and . . <EOS>

> я горжусь своим братом .
= i m proud of my brother .
< i m going my my . <EOS>

> мне будет вас не хватать .
= i am going to miss you .
< i m going to miss you a lot . <EOS>

> мне с трудом в это верится .
= i m having trouble believing it .
< i m having trouble with this myself . <EOS>



In [30]:
hidden_size = 256

encoder_1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder_2 = AttnDecoderRNN_MLP(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder_1, attn_decoder_2, 75000, print_every=5000)

1m 12s (- 16m 57s) (5000 6%) 3.0870
2m 21s (- 15m 19s) (10000 13%) 2.5757
3m 29s (- 13m 58s) (15000 20%) 2.3159
4m 38s (- 12m 45s) (20000 26%) 2.1122
5m 46s (- 11m 33s) (25000 33%) 1.9747
6m 55s (- 10m 23s) (30000 40%) 1.8297
8m 3s (- 9m 12s) (35000 46%) 1.7300
9m 12s (- 8m 3s) (40000 53%) 1.6235
10m 20s (- 6m 53s) (45000 60%) 1.5649
11m 28s (- 5m 44s) (50000 66%) 1.4856
12m 36s (- 4m 35s) (55000 73%) 1.4365
13m 45s (- 3m 26s) (60000 80%) 1.3823
14m 53s (- 2m 17s) (65000 86%) 1.3364
16m 2s (- 1m 8s) (70000 93%) 1.2607
17m 10s (- 0m 0s) (75000 100%) 1.2090


In [31]:
evaluateRandomly(encoder_1, attn_decoder_2)

> они оба очень умны .
= they are both very intelligent .
< they are both very intelligent . <EOS>

> я на нее зол .
= i m angry with her .
< i m angry with her . <EOS>

> эту проблему уладил я .
= i m the one who handled that problem .
< i m the one who chose that . . <EOS>

> вы не умираете .
= you re not dying .
< you re t dying . <EOS>

> ты вольна говорить что думаешь .
= you are free to say what you think .
< you are free to think what you are . <EOS>

> я все еще жду свои деньги .
= i m still waiting for my money .
< i m still waiting for my house . <EOS>

> вы совсем одна .
= you re all alone .
< you re all alone . <EOS>

> мы гордимся тем кто мы есть .
= we re proud of who we are .
< we are proud of our we are . <EOS>

> вы ошибаетесь .
= you re mistaken .
< you re wrong . <EOS>

> я абсолютно серьезен .
= i m completely serious .
< i m completely serious . <EOS>



In [35]:
output_words, attentions = evaluate(
    encoder_1, attn_decoder_2, "она примерно такого же роста что и я .")
plt.matshow(attentions.numpy())
plt.show()

In [36]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder_1, attn_decoder_1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("мы гордимся тем кто мы есть .")

evaluateAndShowAttention("я с тобои работаю .")

evaluateAndShowAttention("мне понадобится еще немного времени .")

evaluateAndShowAttention("она задушила его подушкои .")

input = мы гордимся тем кто мы есть .
output = she is a . . <EOS>
input = я с тобои работаю .
output = he is . <EOS>
input = мне понадобится еще немного времени .
output = you re going to be . <EOS>
input = она задушила его подушкои .
output = you re staying . <EOS>
